In [1]:
# SRC comes from this link: https://fenicsproject.org/pub/tutorial/html/._ftut1008.html
# Version 1.0.0
# 11/23/2021: version 1.0.0. The code can simulate deformation subjuected to far-fielding loading. (D.Liu)
# Elastic deformation from 3D faulting simulated by modulus reduction in the fault zone.
# Created on 11/23/2021 by Dunyu Liu (dliu@ig.utexas.edu)
# Modified some parameters 11/27/2021 by Xu (xiaohua.xu@austin.utexas.edu)
# Added fault mesh refinement 11/30/2021 by Xu and Liu
# Added vp/vs ratio change (lambda change) 12/01/2021 by Xu
# Splitting the code to calculate shear and extension separately 12/05/2021 by Xu
# Further refined the code with secondary refinement near fault 01/25/2022 by Xu
# Adding the V-Shaped fault 02/28/2022 by Xu, probably need mesh changes later
# Adding gmsh input for v-shaped fault 03/21/2022 by Liu and Xu

In [15]:
"""
Linear elastic problem.

  -div(sigma(u)) = f

The model is used to simulate earthquake deformation. 
Faulting is simulated via modulus reduction in the fault zone. 
"""

from __future__ import print_function
from fenics import *
from ufl import nabla_div

# Set path and mesh name
path = "/Users/xiaohua/Documents/softwares/GMesh/mine/elasticity_v/"
mname = "vfault_0.5km"

# pick one mode
#mode = "shear"
mode = "extension"

# Model domain.
#L = 5e3; W = 5e3; Depth = 5e3
n1, n2, n3 = 8, 8, 8
mesh_re = 1
reduction_pct = 0.12 #0.3 #0.06 #0.12 #0.2 #1e-9 #0.32 #0.25 # 0.33 # 0.5 #1 # reduction in shear modulus
#reduction_pct = 1 #
lambda_inc = 1.2 #imulate increased vp/vs ratio in the fault, 8%vp increase ~ 50% lambda increase

# parameters 
# 22e-6 shear 11.7e-6 extension
fudge_fac = 1
fault_zone_width = 0.25e3 # fault zone width
fault_zone_depth = 2e3 # fault zone depth
edge_length = 0 #0.5e3
W = fault_zone_width*32; L = W/2; Depth = W

shear_slip = 22e-6*W/2 * fudge_fac #0.033# far-field loading by slip
normal_slip = 11.7e-6*W/2 * fudge_fac #0.0175

# Rock properties
#vp = 6000
#vs = 3464
rho = 2670
#mu = vs*vs*rho
mu = 10e9
nu = 0.25
# shear_modulus inside the fault zone
mu_fault = mu*reduction_pct

lambda_ = 2*mu*nu/(1-2*nu)
lambda_fault = 2*mu_fault*nu/(1-2*nu) * lambda_inc
g = 0       # gravitiy, assumed to be zero in this application.

dim = 3     # dimension

# Create mesh and define function space
iz = 1
case = 2
if case == 1:
    mesh = BoxMesh(Point(0, 0, 0), Point(L, W, Depth), n1, n2, n3) #,"crossed, left, right")
    if mesh_re == 1:
        for i in range(iz):
            cell_markers = MeshFunction("bool", mesh, dim)
            cell_markers.set_all(False)
            for cell in cells(mesh):
                if cell.midpoint()[0] <= L/2 + fault_zone_width*4 \
                and cell.midpoint()[0] >= L/2 - fault_zone_width*4 \
                and cell.midpoint()[2] >= Depth - 2*fault_zone_depth:
                    cell_markers[cell] = True
            mesh = refine(mesh, cell_markers)
            cell_markers = MeshFunction("bool", mesh, dim)
            cell_markers.set_all(False)
            for cell in cells(mesh):
                if cell.midpoint()[0] <= L/2 + fault_zone_width*3 \
                and cell.midpoint()[0] >= L/2 - fault_zone_width*3 \
                and cell.midpoint()[2] >= Depth - 2*fault_zone_depth:
                    cell_markers[cell] = True
            mesh = refine(mesh, cell_markers)
        for i in range(iz):
            cell_markers = MeshFunction("bool", mesh, dim)
            cell_markers.set_all(False)
            for cell in cells(mesh):
                if cell.midpoint()[0] <= L/2 + fault_zone_width*2 \
                and cell.midpoint()[0] >= L/2 - fault_zone_width*2 \
                and cell.midpoint()[1] >= edge_length and cell.midpoint()[1] < W - edge_length \
                and cell.midpoint()[2] >= Depth - 1.5*fault_zone_depth:
                    cell_markers[cell] = True
            mesh = refine(mesh, cell_markers)
            cell_markers = MeshFunction("bool", mesh, dim)
            cell_markers.set_all(False)
            for cell in cells(mesh):
                if cell.midpoint()[0] <= L/2 + fault_zone_width \
                and cell.midpoint()[0] >= L/2 - fault_zone_width \
                and cell.midpoint()[1] >= edge_length and cell.midpoint()[1] < W - edge_length \
                and cell.midpoint()[2] >= Depth - fault_zone_depth:
                    cell_markers[cell] = True
            mesh = refine(mesh, cell_markers) 
elif case == 2:
# Load mesh   
    mesh = Mesh(path + mname + '.xml')
    boundaries = MeshFunction("size_t", mesh, path + mname + '_facet_region.xml')
    mf = MeshFunction("size_t", mesh, path + mname + '_physical_region.xml')
        
#    for i in range(iz):
#        cell_markers = MeshFunction("bool", mesh, dim)
#        cell_markers.set_all(False)
#        for cell in cells(mesh):
#            if 2*(cell.midpoint()[0] - L/2 - fault_zone_width)*fault_zone_depth/fault_zone_width \
#                + (Depth - fault_zone_depth) <= cell.midpoint()[2] \
#            and -2*(cell.midpoint()[0] - L/2 + fault_zone_width)*fault_zone_depth/fault_zone_width \
#                + (Depth - fault_zone_depth) <= cell.midpoint()[2]:
#                cell_markers[cell] = True
#        mesh = refine(mesh, cell_markers)
#        cell_markers = MeshFunction("bool", mesh, dim)
#        cell_markers.set_all(False)
#        for cell in cells(mesh):
#            if 2*(cell.midpoint()[0] - L/2 - fault_zone_width/2)*fault_zone_depth/fault_zone_width \
#                + (Depth - fault_zone_depth) <= cell.midpoint()[2] \
#            and -2*(cell.midpoint()[0] - L/2 + fault_zone_width/2)*fault_zone_depth/fault_zone_width \
#                + (Depth - fault_zone_depth) <= cell.midpoint()[2]:
#                cell_markers[cell] = True
#        mesh = refine(mesh, cell_markers)
             
V = VectorFunctionSpace(mesh, 'P', 1)
#boundaries = MeshFunction("size_t", mesh, dim-1)

tol = 1E-14

# Define boundary condition
class Top_boundary(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[2], Depth, DOLFIN_EPS) and on_boundary
class Bottom_boundary(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[2], 0.0, DOLFIN_EPS) and on_boundary
class Left_boundary(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[0], 0, DOLFIN_EPS) and on_boundary
class Right_boundary(SubDomain):
    def inside(self, x, on_boundary):   
        return near(x[0], L, DOLFIN_EPS) and on_boundary
class North_boundary(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[1], W, DOLFIN_EPS) and on_boundary
class South_boundary(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[1], 0, DOLFIN_EPS) and on_boundary    
#boundaries.set_all(0)
#Top_boundary().mark(boundaries, 1)
#Bottom_boundary().mark(boundaries, 2)
#Left_boundary().mark(boundaries, 3)
#Right_boundary().mark(boundaries, 4)
#North_boundary().mark(boundaries, 5)
#South_boundary().mark(boundaries, 6)
# Rename boundaries
top = 1
bottom = 91
left = 89
right = 90
north = 5
south = 6
# setting boundary conditions
#bc = [DirichletBC(V.sub(2), Constant((0.0)), boundaries, bottom),
#        DirichletBC(V.sub(0), Constant((0.0)), boundaries, left),
#        DirichletBC(V.sub(0), Constant((0.0)), boundaries, right),
#        DirichletBC(V.sub(1), shear_slip, boundaries, left), # far-field loading via slip
#        DirichletBC(V.sub(1), -shear_slip, boundaries, right)] # far-field loading via slip
if mode == "shear":
# shear
    bc = [  DirichletBC(V.sub(0), Constant((0.0)), boundaries, left),
        DirichletBC(V.sub(0), Constant((0.0)), boundaries, right),
#        DirichletBC(V.sub(2), Constant((0.0)), boundaries, left),
#        DirichletBC(V.sub(2), Constant((0.0)), boundaries, right),
        DirichletBC(V.sub(1), Constant((0.0)), boundaries, left), # far-field loading via slip
        DirichletBC(V.sub(1), -2*shear_slip, boundaries, right), # far-field loading via slip
        DirichletBC(V.sub(2), Constant((0.0)), boundaries, bottom)] #,
#        DirichletBC(V.sub(0), -normal_slip, boundaries, left),
#        DirichletBC(V.sub(0), normal_slip, boundaries, right)]
    
if mode == "extension":
# extension
    bc = [  DirichletBC(V.sub(0), -normal_slip, boundaries, left),
        DirichletBC(V.sub(0), normal_slip, boundaries, right),
        DirichletBC(V.sub(2), Constant((0.0)), boundaries, bottom)] 

#bc = [DirichletBC(V.sub(2), Constant((0.0)), boundaries, bottom),
#        DirichletBC(V.sub(0), Constant((0.0)), boundaries, left),
#        DirichletBC(V.sub(0), normal_slip, boundaries, right),
#        DirichletBC(V.sub(1), Constant((0.0)), boundaries, left), # far-field loading via slip
#        DirichletBC(V.sub(1), Constant((0.0)), boundaries, right)] # far-field loading via slip


# Define fault zone as subdomains.
# At the center of x axis.
#class Omega_0(SubDomain):
#    def inside(self, x, on_boundary):
#        return 2*(x[0] - L/2)*fault_zone_depth/fault_zone_width + (Depth - fault_zone_depth) -20 <= x[2] and \
#            -2*(x[0] - L/2)*fault_zone_depth/fault_zone_width + (Depth - fault_zone_depth) -20 <= x[2]
      
#mf = MeshFunction("size_t", mesh, 3)
#subdomain0 = Omega_0()
#subdomain1 = Omega_1()
#mf.set_all(10)
#subdomain0.mark(mf,11) # 1, the middle anisotrpic layer.
rock = 92
fault = 93

# Define strain and stress
def epsilon(u):
    return 0.5*(nabla_grad(u) + nabla_grad(u).T)
    #return sym(nabla_grad(u))

def sigma(u):
    return lambda_mod*nabla_div(u)*Identity(d) + 2*shear_mod*epsilon(u)

# Define class to set different parameters to different subdomains
class K(UserExpression):
            def __init__(self, subdomains, k_fault, k_rock, **kwargs):
                super().__init__(**kwargs)
                self.subdomains = subdomains
                self.k_fault = k_fault
                self.k_rock = k_rock

            def eval_cell(self, values, x, cell):
                if self.subdomains[cell.index] == fault: #anisotrpic layer
                    values[0] = self.k_fault
                elif self.subdomains[cell.index] == rock:
                    values[0] = self.k_rock
            def values_shape(self):
                return (1,)            
shear_mod = K(mf, mu_fault, mu, degree=0)
lambda_mod = K(mf,lambda_fault, lambda_, degree=0)

# Define variational problem
u = TrialFunction(V)
d = u.geometric_dimension()  # space dimension
v = TestFunction(V)
f = Constant((0, 0, -rho*g))
T = Constant((0, 0, 0))
a = inner(sigma(u), epsilon(v))*dx
b = inner(sigma(u), epsilon(v))*dx
L = dot(f, v)*dx + dot(T, v)*ds

# Compute solution
#u = Function(V)
#solve(a == L, u, bc)

A, bb = assemble_system(a, L, bc)
#A, bb = assemble_system(a, L)
#for bc in bcs:
#    bc.apply(A)
#    bc.apply(bb)
# Assemble preconditioner system
P, btmp = assemble_system(b, L, bc)
#P, btmp = assemble_system(b, L)
#for bc in bcs:
#    bc.apply(P)
print('Assembling the system and preconditioner ...')
 
# Create Krylov solver and AMG preconditioner
  
#solver = KrylovSolver("minres", "amg") # best.
solver = KrylovSolver("gmres", "amg") # second best.
#solver = KrylovSolver("tfqmr", "amg") # least
 
solver.parameters["relative_tolerance"] = 1.0e-9   # most important
solver.parameters["absolute_tolerance"] = 1.0e-15  # set a small number
solver.parameters["divergence_limit"] = 1.0e4
solver.parameters["maximum_iterations"] = 7000
solver.parameters["error_on_nonconvergence"] = True
solver.parameters["nonzero_initial_guess"] = False
solver.parameters["report"] = True
solver.parameters["monitor_convergence"] = True
# Associate operator (A) and preconditioner matrix (P)
#solver.set_operators(A, P)
solver.set_operators(A, P)

# Solve
u = Function(V)
solver.solve(u.vector(), bb)
print('Solving the system and writing out the results ...')


# Plot solution
#plot(u, title='Displacement', mode='displacement')

# Plot stress
s = sigma(u) - (1./3)*tr(sigma(u))*Identity(d)  # deviatoric stress
von_Mises = sqrt(3./2*inner(s, s))
V = FunctionSpace(mesh, 'P', 1)
von_Mises = project(von_Mises, V)
#plot(von_Mises, title='Stress intensity')

# Compute magnitude of displacement
u_magnitude = sqrt(dot(u, u))
u_magnitude = project(u_magnitude, V)
#plot(u_magnitude, 'Displacement magnitude')
#print('min/max u:',
#      u_magnitude.vector().array().min(),
#      u_magnitude.vector().array().max())

# Save solution to file in VTK format
if mode == "shear":
    File('elasticity/displacement_shear_v.pvd') << u
if mode == "extension":
    File('elasticity/displacement_extension_v.pvd') << u
#File('elasticity/von_mises.pvd') << von_Mises
#File('elasticity/magnitude.pvd') << u_magnitude

# Hold plot
#interactive()
print('Simulation Done')
#reset -f

Assembling the system and preconditioner ...
Solving linear system of size 77634 x 77634 (PETSc Krylov solver).Solving the system and writing out the results ...

  0 KSP preconditioned resid norm 2.988591194691e+00 true resid norm 1.847889502939e+13 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP preconditioned resid norm 5.414573992874e-01 true resid norm 3.640147277460e+12 ||r(i)||/||b|| 1.969894450761e-01
  2 KSP preconditioned resid norm 2.176229245278e-01 true resid norm 1.629727565648e+12 ||r(i)||/||b|| 8.819399444911e-02
  3 KSP preconditioned resid norm 4.085693312669e-02 true resid norm 2.945384801694e+11 ||r(i)||/||b|| 1.593918249446e-02
  4 KSP preconditioned resid norm 7.010426840486e-03 true resid norm 3.661504661679e+10 ||r(i)||/||b|| 1.981452167922e-03
  5 KSP preconditioned resid norm 1.381166380080e-03 true resid norm 8.588094387484e+09 ||r(i)||/||b|| 4.647515110521e-04
  6 KSP preconditioned resid norm 2.859145564052e-04 true resid norm 1.562366618268e+09 ||r(i)||/||b|| 8.

In [5]:
reset -f